In [38]:
import ROOT as r
import math as m
import os

In [39]:
Output = r.TChain("output")
directory = "/global/projecta/projectdirs/atlas/eressegu/Ntuple_h026_0630/data15/data/data15/root"
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        Output.Add(f)

i = 0
Output2 = r.TChain("output")
directory2 = "/global/projecta/projectdirs/atlas/eressegu/Ntuple_h026_0630/NonRes/mc16a/NonRes/root"
for filename in os.listdir(directory2):
    i += 1
    if i == 20:
        break
    f = os.path.join(directory2, filename)
    if os.path.isfile(f):
        Output2.Add(f)

In [40]:
photon1 = r.TLorentzVector()
photon2 = r.TLorentzVector()
diphoton = r.TLorentzVector()

h_2015_thetaphi = r.TH2F("2015thetavsphi", "2015thetavsphi", 60,-180, 180, 60, -90, 90)
h_MC_thetaphi = r.TH2F("MCthetavsphi", "MCthetavsphi", 60,-180, 180, 60, -90, 90)
h_MC_M = r.TH1F("h_MC_M", "h_MC_M", 60, 0, 600)
h_2015_M = r.TH1F("h_2015_M", "h_2015_M", 60, 0, 600)

Warning in <TROOT::Append>: Replacing existing TH1: 2015thetavsphi (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: 2015thetavsphi (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: MCthetavsphi (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: MCthetavsphi (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_MC_M (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_2015_M (Potential memory leak).


In [41]:
for entry in Output:
    if entry.isPassed:
        photon1.SetPtEtaPhiE(entry.ph_pt1, entry.ph_eta1, entry.ph_phi1, entry.ph_E1)
        photon2.SetPtEtaPhiE(entry.ph_pt2, entry.ph_eta2, entry.ph_phi2, entry.ph_E2)
        diphoton = photon1 + photon2
        angle = diphoton.Pt() / diphoton.Pz()
        actual_angle = m.atan(angle) * 180 / m.pi
        phi = diphoton.Phi() * 180 / m.pi
        h_2015_thetaphi.Fill(phi, actual_angle)
        h_2015_M.Fill(diphoton.M())

In [42]:
for entry in Output2:
    if entry.isPassed:
        photon1.SetPtEtaPhiE(entry.ph_pt1, entry.ph_eta1, entry.ph_phi1, entry.ph_E1)
        photon2.SetPtEtaPhiE(entry.ph_pt2, entry.ph_eta2, entry.ph_phi2, entry.ph_E2)
        diphoton = photon1 + photon2
        angle = diphoton.Pt() / diphoton.Pz()
        actual_angle = m.atan(angle) * 180 / m.pi
        phi = diphoton.Phi() * 180 / m.pi
        h_MC_thetaphi.Fill(phi, actual_angle)
        h_MC_M.Fill(diphoton.M())

In [43]:
fit = r.TF1("expofit", "expo", 100, 150)
h_2015_M.Fit(fit, "S", "", 100, 120)

fit2 = r.TF1("expofit", "expo", 100, 150)
h_MC_M.Fit(fit2, "S", "", 100, 120)

 FCN=5.72212e-28 FROM MIGRAD    STATUS=CONVERGED      28 CALLS          29 TOTAL
                     EDM=4.33155e-22    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     4.24959e+00   2.56490e-01   0.00000e+00   4.76575e-09
   2  Slope        4.11574e-02   2.30817e-03  -0.00000e+00   5.31081e-07
 FCN=2.49042e-25 FROM MIGRAD    STATUS=CONVERGED      28 CALLS          29 TOTAL
                     EDM=3.72625e-18    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     8.26038e+00   3.50247e-02  -0.00000e+00   1.74157e-07
   2  Slope        4.49742e-02   3.14934e-04   0.00000e+00   7.11385e-06


In [44]:
N1_2015 = fit.Integral(100, 120)
print(N1_2015)
N3_2015 = fit.Integral(130, 150)
print(N3_2015)
N2_2015 = fit.Integral(120, 130)
print(N2_2015)
N1_MC = fit2.Integral(100, 120)
print(N1_MC)
N3_MC = fit2.Integral(130, 150)
print(N3_MC)
N2_MC = fit2.Integral(120, 130)
print(N2_MC)

133346.53532574078
458367.9123969992
121042.07471890091
11259909.681165075
43400649.354421765
10779445.222831968


In [45]:
scale_MC = (N1_2015 + N3_2015) / (N1_MC + N3_MC)
print(scale_MC)

0.010825254226498184


In [46]:
h_2015_M_clone = h_2015_M.Clone("mass_2015_clone")
h_MC_M_clone = h_MC_M.Clone("mass_MC_clone")
h_MC_M_clone.Scale(scale_MC)
canvas_1 = r.TCanvas("higgs_mass_both", "higgs_mass_both")
h_2015_M_clone.Draw()
h_MC_M_clone.Draw("SAME")
h_2015_M_clone.GetXaxis().SetTitle("m_{#gamma#gamma} [GeV]")
h_2015_M_clone.GetYaxis().SetTitle("Entries")
canvas_1.SaveAs("MC2015Masses.png")

canvas_2 = r.TCanvas("mass_subtracted", "mass_subtracted")
h_2015_M_clone.Add(h_MC_M_clone, -1)
h_2015_M_clone.Draw()
h_2015_M_clone.SetLineWidth(2)
h_2015_M_clone.SetLineColor(2)
h_2015_M_clone.SetLineStyle(1)
h_2015_M_clone.SetFillColor(2)
canvas_2.SaveAs("MC2015subtracted.png")

Warning in <TCanvas::Constructor>: Deleting canvas with same name: higgs_mass_both
Info in <TCanvas::Print>: png file MC2015Masses.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: mass_subtracted
Info in <TCanvas::Print>: png file MC2015subtracted.png has been created


In [47]:
canvas_thetaphi = r.TCanvas("higgs_thetaphi", "higgs_thetaphi")
h_2015_thetaphi.Draw("colz")
h_2015_thetaphi.GetXaxis().SetTitle("#phi_{#gamma#gamma}")
h_2015_thetaphi.GetYaxis().SetTitle("#theta_{#gamma#gamma}")
canvas_thetaphi.SaveAs("2015thetaphi.png")

h_MC_thetaphi.Draw("colz")
h_MC_thetaphi.GetXaxis().SetTitle("#phi_{#gamma#gamma}")
h_MC_thetaphi.GetYaxis().SetTitle("#theta_{#gamma#gamma}")
canvas_thetaphi.SaveAs("MCthetaphi.png")

h_2015_thetaphi_clone = h_2015_thetaphi.Clone()
h_MC_thetaphi_clone = h_MC_thetaphi.Clone()
h_MC_thetaphi_clone.Scale(scale_MC)
h_2015_thetaphi_clone.Add(h_MC_thetaphi_clone, -1)
h_2015_thetaphi_clone.Draw("colz")
h_2015_thetaphi_clone.GetXaxis().SetTitle("#phi_{#gamma#gamma}")
h_2015_thetaphi_clone.GetYaxis().SetTitle("#theta_{#gamma#gamma}")
canvas_thetaphi.SaveAs("MC2015thetaphisubtracted.png")


Warning in <TCanvas::Constructor>: Deleting canvas with same name: higgs_thetaphi
Info in <TCanvas::Print>: png file 2015thetaphi.png has been created
Info in <TCanvas::Print>: png file MCthetaphi.png has been created
Info in <TCanvas::Print>: png file MC2015thetaphisubtracted.png has been created


In [48]:
canvas_3 = r.TCanvas("mass", "mass")
h_2015_M.SetLineWidth(2)
h_2015_M.SetLineColor(2)
h_2015_M.SetLineStyle(1)
h_2015_M.SetFillColor(2)
h_2015_M.Draw()
h_2015_M.GetXaxis().SetTitle("m_{#gamma#gamma} [GeV]")
h_2015_M.GetYaxis().SetTitle("Entries")
canvas_3.SaveAs("2015_M.png")

canvas_4 = r.TCanvas("mass", "mass")
h_MC_M.SetLineWidth(2)
h_MC_M.SetLineColor(2)
h_MC_M.SetLineStyle(1)
h_MC_M.SetFillColor(2)
h_MC_M.Draw()
h_MC_M.GetXaxis().SetTitle("m_{#gamma#gamma} [GeV]")
h_MC_M.GetYaxis().SetTitle("Entries")
canvas_4.SaveAs("MC_M.png")

Warning in <TCanvas::Constructor>: Deleting canvas with same name: mass
Info in <TCanvas::Print>: png file 2015_M.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: mass
Info in <TCanvas::Print>: png file MC_M.png has been created
